# Comparar catálogo

## Parámetros

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Catalog/Decathlon'
FILE_MX = 'Decathlon_MX.csv'
FILE_CL = 'Decathlon_CL.csv'

## Clase

In [2]:
# Control de datos
from time import sleep
from pathlib import Path
from IPython.display import clear_output

# Ingeniería de variables
from math import sqrt
from numpy import nan
from collections import Counter
from unicodedata import normalize
from re import sub, findall, UNICODE
from difflib import get_close_matches
from pandas import DataFrame, read_csv, options
options.mode.chained_assignment = None

class CompareCatalog:
    def __init__(self, base_dir:str, file_mx: str, file_cl: str) -> None:
        '''
        Obtener un directorio como texto y convertirlo a tipo Path para unir directorios, buscar archivos, etc.
        '''
        self.base_dir = Path(base_dir)
        # Definir la ruta completa para leer cada archivo
        self.file_mx = self.base_dir.joinpath(file_mx)
        self.file_cl = self.base_dir.joinpath(file_cl)
        # Verificar que existe el archivo en el directorio
        for file_path in [self.file_mx, self.file_cl]:
            if not file_path.is_file():
                file_name = ''.join(file_path.split('/')[-1])
                print(f'Debería haber un archivo llamado: {file_name} en:\n{self.base_dir}\n\nAgrega este archivo e intenta de nuevo!\n')


    def cool_print(self, text: str, sleep_time: float=0.01) -> None: 
        '''
        Imprimir como si se fuera escribiendo
        '''
        acum = ''
        for x in text: 
            # Acumular texto
            acum += x
            # Limpiar pantalla
            clear_output(wait=True)
            # Esperar un poco para emular efecto de escritura
            sleep(sleep_time)
            # Imprimir texto acumulado
            print(acum)
        # Mantener el texto en pantalla
        sleep(1)


    def get_csv(self, file_path, **kwargs) -> DataFrame: 
        '''
        Obtener tabla a partir de un archivo .csv
        '''
        file_name = ''.join(str(file_path).split('/')[-1])
        try: 
            df = read_csv(file_path, low_memory=False, **kwargs)
            # Obtener el número de renglones y columnas para informar al usuario
            df_shape = df.shape
            self.cool_print(f'Archivo con nombre {file_name} fue encontrado en:\n{self.base_dir}\nCon {df_shape[0]} renglones y {df_shape[-1]} columnas')
            df.columns = map(lambda x: str(x).strip().replace(' ','_').lower(), df.columns)
            return df
        # Informar que hubo error al intentar importar el csv
        except: self.cool_print(f'No se encontró el archivo con nombre {file_name} en:\n{self.base_dir}\nSi el archivo csv existe, seguramente tiene un encoding y/o separador diferente a "utf-8" y "," respectivamente\nIntenta de nuevo!')
    

    def export_csv(self, df: DataFrame, file_name: str, name_suffix=None, **kwargs) -> None: 
        '''
        Exportar un archivo en formato csv
        '''
        export_name = f'{file_name}.csv' if name_suffix==None else f'{file_name}_{name_suffix}.csv'
        df.to_csv(self.base_dir.joinpath(export_name), **kwargs)
        self.cool_print(f'Archivo: {export_name} fue exportado exitosamente en:\n{self.base_dir}')


    def clean_text(self, text: str, pattern: str="[^a-zA-Z0-9\s]", lower: bool=False) -> str: 
        '''
        Limpieza de texto
        '''
        # Reemplazar acentos: áàäâã --> a
        clean = normalize('NFD', str(text).replace('\n', ' \n ')).encode('ascii', 'ignore')
        # Omitir caracteres especiales !"#$%&/()=...
        clean = sub(pattern, ' ', clean.decode('utf-8'), flags=UNICODE)
        # Mantener sólo un espacio
        clean = sub(r'\s{2,}', ' ', clean.strip())
        # Minúsculas si el parámetro lo indica
        if lower: clean = clean.lower()
        # Si el registro estaba vacío, indicar nulo
        if clean in ('','nan'): clean = nan
        return clean


    def get_cosine(self, vec1, vec2) -> float:
        intersection = set(vec1.keys()) & set(vec2.keys())
        numerator = sum([vec1[x] * vec2[x] for x in intersection])

        sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
        sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
        denominator = sqrt(sum1) * sqrt(sum2)

        if not denominator:return 0.0
        else: return float(numerator) / denominator

    def text_to_vector(self, text: str) -> Counter:
        words = findall(r"\w+", text)
        return Counter(words)

    def cosine_sim(self, text_one: str, text_two: str) -> float:
        vector1 = self.text_to_vector(text_one)
        vector2 = self.text_to_vector(text_two)
        return self.get_cosine(vector1, vector2)

    def choose_correct(self, df: DataFrame, col: str, correct_list: list, suffix: str='correct', fill_value: str='DESCONOCIDO', **kwargs) -> DataFrame:
        '''
        Recibe un DataFrame y una lista de posibilidades, especificando la columna a revisar
        elige la opción que más se parezca a alguna de las posibilidades
        '''
        correct_list = list(set(correct_list))
        # Aplicar limpieza de texto a la lista de posibilidades
        correct_clean = list(map(lambda x: self.clean_text(x, lower=True), correct_list))
        # Hacer un diccionario de posibilidades limpias y las originales recibidas
        correct_dict = dict(zip(correct_clean, correct_list))

        # Aplicar la limpieza a la columna especificada
        df[f'{col}_{suffix}'] = df[col].map(lambda x: self.clean_text(x,lower=True))
        # Encontrar las posibilidades más parecidas
        df[f'{col}_{suffix}'] = df[f'{col}_{suffix}'].map(lambda x: get_close_matches(x, correct_clean, **kwargs))
        # Si existen parecidas, traer la primera opción que es la más parecida
        df[f'{col}_{suffix}'] = df[f'{col}_{suffix}'].map(lambda x: x[0] if isinstance(x,list) and len(x)>0 else nan)
        # Regresar del texto limpio a la posibilidad original, lo no encontrado se llena con "fill_value"
        df[f'{col}_{suffix}'] = df[f'{col}_{suffix}'].map(correct_dict).fillna(fill_value)
        return df


    def compare_catalog(self, id_cols: list=['sku','barcodes'], name_cols: list=['name'], export_result: bool=True) -> DataFrame:
        '''
        Compara dos catálogos empezando con coincidencia exacta de "id_cols" y después coincidencia aproximada de "name_cols"
        '''
        # Importa ambos catálogos
        df_mx = self.get_csv(self.file_mx)[id_cols+name_cols]
        df_cl = self.get_csv(self.file_cl)[id_cols+name_cols]

        # Variables auxiliares para acumular resultado
        df = DataFrame()
        to_omit = []
        for col in id_cols+name_cols:
            # Omitir índices que ya se han encontrado
            without_omit = df_mx.loc[~df_mx.index.isin(to_omit),:].copy()
            if col in name_cols: 
                # Encuentra el texto más parecido del 2o catálogo vs el 1er catálogo
                df_cl_copy = self.choose_correct(df_cl.copy(), col, correct_list=without_omit[col], n=1, cutoff=0.85)
                # Guarda el nombre más parecido de 2 pero va a unir con el nombre de 1
                df_cl_copy.rename({col:f'found_by_{col}',f'{col}_correct':col}, axis=1, inplace=True)
            
            # Renombra la variable con la que unirá
            try: aux = df_cl_copy.rename({col:f'{col}_found'}, axis=1)
            except: aux = df_cl.rename({col:f'{col}_found'}, axis=1)
            # Indica por qué variable fue encontrado dicho registro
            aux['was_found_by'] = col

            # Une ambos catálogos 
            to_append = without_omit.merge(aux, left_on=col, right_on=f'{col}_found', suffixes=('','_found'))

            # Apila el resultado de la variable analizada
            df = df.append(to_append, ignore_index=False)
            # Agrega los índices que deberá omitir en la siguiente iteración
            to_omit.append(to_append.index)

        for col in name_cols:
            # Reemplazar el nombre que más se parecía
            aux = []
            for found_by, original, found in zip(df['was_found_by'], df[f'{col}_found'], df[f'found_by_{col}']):
                if found_by == col: aux.append(found)
                else: aux.append(original)
            df[f'{col}_found'] = aux

            # Omitir la columna para mantener la estructura
            df.drop(f'found_by_{col}', axis=1, inplace=True)

            # Obtener el parecido del nombre original vs el encontrado
            df[f'{col}_similarity'] = df[[col,f'{col}_found']].apply(lambda x: self.cosine_sim(x[0],x[-1]), axis=1)

        # Omitir duplicados
        df = df.sort_values([id_cols[0],f'{name_cols[0]}_found']).drop_duplicates(id_cols[0])

        # Exportar el resultado
        if export_result: self.export_csv(df, file_name='Decathlon', index=False, sep='\t', encoding='utf-16')
        return df

## Transformar

In [3]:
df = CompareCatalog(BASE_DIR, FILE_MX, FILE_CL).compare_catalog()
df.sample(7)

Archivo: Decathlon.csv fue exportado exitosamente en:
/Users/efraflores/Desktop/EF/Corner/Catalog/Decathlon


,sku,barcodes,name,sku_found,barcodes_found,name_found,was_found_by,name_similarity
897,1273231,3.583789e+12,Botas Caza Solognac Renfort 500 Verde Solognac,1273231,3.583790e+12,Bota De Caza Renfort 500,sku,0.447214
8374,631234,3.608460e+12,Panty Bikini Surf Olaian Lazos Sofy Negra Muje...,631234,3.608460e+12,Parte Inferior De Bikini De Surf Mujer Anudada...,sku,0.311400
533,2413581,3.583788e+12,Balón De Balonmano H100 Soft T1 Azul Y Amarillo,2413581,3.583790e+12,Balon De Handball H100 Soft T1,sku,0.544331
12568,2505923,3.608430e+12,Tenis De Running Hombre Run Active Azul Oscuro...,2505923,3.608430e+12,Zapatillas De Running Para Hombre Run Active,sku,0.629941
5005,2665416,3.583788e+12,Guantes Portero De Fútbol Kipsta F500 Niños Az...,2665416,3.583790e+12,Guantes Arquero De Futbol F500 Ninos,sku,0.339683
14975,2777020,3.608410e+12,"Xc 100 29"" Rr Cn Fr Rockrider",2777020,3.608410e+12,Xc 100 29 Rr Cn Fr,sku,0.925820
6042,2342398,NaN,Mallas Largas Atletismo Hombre Run Warm Negro,2342398,3.608430e+12,Calzas Largas Running Run Warm Hombre,sku,0.617213
